In [1]:
import dash                     # pip install dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px     # pip install plotly==5.2.2

import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime
# import timestamp
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures

import plotly.express as px

<ipython-input-1-8c6bdfd9b70d>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-8c6bdfd9b70d>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [13]:

class Data():
    def __init__(self, path = None):
        
        self.directory = (glob.glob("/Data*merged.csv")) 
        # Add All the functionality to the class for organisation
        self.dailyNames = (glob.glob("daily*merged.csv")) 


        self.init()
        self.pdfs = []
        self.dailies = {}
    def init(self):
        for idx, dataName in enumerate(self.directory):
            self.pdfs.append(self.getDF(-1,dataName))

        for idx, data in enumerate(self.dailyNames):
            self.dailies[data]  =self.getDF(data)


    def getDF(self, i, byname =0):
        if(byname != 0):
            return(pd.read_csv(byname))
        return(pd.read_csv(self.directory[i]))





def containTimeInstant():
    timeList = ["time, "]

def read_file(path, expend_print = False, convertTimeDate = False):
    """ 
    read files
    """
    # formatting to pandas
    pd.Timestamp('2014-01-23 00:00:00', tz=None).to_pydatetime()

    print(path)
    
    # read csv
    df  = pd.read_csv(path)
    if(expend_print):
        print(path)
        print(f"Size: {df.shape}")
        print(df.head(10))
        print(df.columns)
        print((df.Id.unique()))
        print((list(df.columns)))
    df.dropna(subset=list(df.columns), how='any', axis=0).reset_index(inplace = True)

    if(convertTimeDate):
        pass

    ID = "Id"
    if ID in list(df.columns):
        # df['date_column'] = pd.to_datetime(df['datetime_column']).dt.date
        return df
    return None


def saved_file(df, name ):
    df.to_csv(f'{name}.csv')

def merge_data_to_one(files, safefile = False, saveFileName = "OutputMerged"):
    outputMerged = pd.DataFrame()
    
    for idx, fileName in enumerate(set(files)):
        data = read_file(fileName)
        if isinstance(data, pd.DataFrame):
            print(fileName)
            if(idx<1):
                outputMerged = data
            else:
                pd.merge(outputMerged, data, on = 'Id', how="left")
    
    if safefile: saved_file(outputMerged, saveFileName)


def compareSimilarity(A, B):
    A.sort()
    B.sort()

    max_val = max([len(A),len(B)])

    if max_val == len(A):arrMax, secondMax =A, B 
    else: arrMax, secondMax =A, B = A, B 
    
    sumCountSimilarities = 0

    sum_=0
    for i in list(arrMax):
        for j in list(secondMax):
            if i==j:
                if i<j : break
                sumCountSimilarities+=1
                break
             
    return (sumCountSimilarities, sumCountSimilarities/max_val)


def compareamountof(lists):
    l = len(lists)
    output = []

    for i in range(l):
        for j in range(i, l):
            A = read_file(lists[i])
            B = read_file(lists[j])


            similarities, ratiosim = compareSimilarity(A.Id.unique(), B.Id.unique())
            output.append([lists[i], lists[j],similarities, ratiosim])

    return output


def divideInsertTime(data, colName):
    # time_col_split = data.colName.str.split(" ")
    # split_amount = len(list(time_col_split[0]))

    date = pd.to_datetime(data[colName])
    try:
        Id_To_Idx = {id :num for num,id in enumerate(Heart_By_Idx.Id.unique())}
        Heart_By_Idx['PersonID'] = data.Id.map(Id_To_Idx)

        data["Hour"] = date.dt.hour
        data["Day"]  = date.dt.day 
        data["Minute"] = date.dt.minute
        data["DateOfWeek"] = date.dt.day_name()      
        print("Date Changed")
    except:
        e = sys.exc_info()
        print(f"Could Not Prase Data: Check Col Name; Erorr: {sys.exc_info()}")        

    return data




def pvalue_test(data, col_name, based_on_col_vals = None, tresh = 0.05):
    i = 0
    significant = []
    if based_on_col_vals ==None:
        print(data.columns)
        based_on_col_vals = data[col_name]
    print(f"Based on column: {col_name}")
    for (columnName, columnData) in data.iteritems():
        if i>1:
            stat, p = kendalltau(columnData, based_on_col_vals)
            print('Column Name : ', columnName)
            print(p)
            if(p<tresh):
                print(f"Significant - {p} - to {col_name} with {columnName}")
                significant.append((p,col_name, columnName))
        i +=1
    return significant


def sliceDataBasedonDescribeval(data, col,dateCol, tresh = 5):
    dis = data.describe()
    list_col = list((((dis.loc[col][dis.loc['std']>tresh]))).index)
    list_col.append(dateCol)
    
    data = data[list_col]
    data['Time_str'] = df[dateCol].astype('str')
    

    return data

#TODO : Set Index: data.set_index("Id")

In [22]:
data_csvs = (glob.glob("../Data/*merged.csv")) 
dailyData = (glob.glob("../Data/daily*merged.csv"))  +  (glob.glob("../Data/heartrate*merged.csv")) 

data_csvs

['../Data\\dailyActivity_merged.csv',
 '../Data\\dailyCalories_merged.csv',
 '../Data\\dailyIntensities_merged.csv',
 '../Data\\dailySteps_merged.csv',
 '../Data\\heartrate_seconds_merged.csv',
 '../Data\\hourlyCalories_merged.csv',
 '../Data\\hourlyIntensities_merged.csv',
 '../Data\\hourlySteps_merged.csv',
 '../Data\\minuteCaloriesNarrow_merged.csv',
 '../Data\\minuteCaloriesWide_merged.csv',
 '../Data\\minuteIntensitiesNarrow_merged.csv',
 '../Data\\minuteIntensitiesWide_merged.csv',
 '../Data\\minuteMETsNarrow_merged.csv',
 '../Data\\minuteSleep_merged.csv',
 '../Data\\minuteStepsNarrow_merged.csv',
 '../Data\\minuteStepsWide_merged.csv',
 '../Data\\sleepDay_merged.csv',
 '../Data\\weightLogInfo_merged.csv']

In [87]:
# Data: https://www.dallasopendata.com/Services/Participants-Inventory/qgg6-h4bd

df = pd.read_csv(data_csvs[0])
df.ActivityDate = pd.to_datetime(df.ActivityDate)
df['Time_str'] = df.ActivityDate.astype('str')
# df["intake_time"] = df["intake_time"].dt.hour
# print(df.head())
df  =sliceDataBasedonDescribeval(df, 'std','ActivityDate')

df.head()


<ipython-input-13-825045f40750>:166: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,ActivityDate,Time_str
0,1503960366,13162,25,13,328,728,1985,2016-04-12,2016-04-12
1,1503960366,10735,21,19,217,776,1797,2016-04-13,2016-04-13
2,1503960366,10460,30,11,181,1218,1776,2016-04-14,2016-04-14
3,1503960366,9762,29,34,209,726,1745,2016-04-15,2016-04-15
4,1503960366,12669,36,10,221,773,1863,2016-04-16,2016-04-16


In [ ]:
# compile the list of dataframes you want to merge
data_frames = [heart_rate_std, heart_rate_mean, heart_rate_median]
nan_value = 0

result_1 = pd.concat(data_frames, join='outer', axis=1).fillna(nan_value)

result_2 = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='outer'), 
                  data_frames).fillna(nan_value)

arr_idx = np.array([(list(values)[0],  list(values)[1]) for values in (result_2.index)])

result_2["Date"] = arr_idx[:,0]
result_2["Id"] = arr_idx[:,1]
result_2.rename(columns = {"Value_x": "Std","Value_y": "Mean", "Value": "Mediam"}, inplace = True)

Dropeed_Indx_HeartRate = result_2.reset_index(drop=True)



data_frames2 = [Dropeed_Indx_HeartRate, meged_cl_sle_steps]
result_3 = pd.concat(data_frames2, join='outer', axis=0).fillna(nan_value)

result_4 = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='outer'), 
                  data_frames).fillna(nan_value)


In [89]:
df.ActivityDate.apply(lambda x: x.strftime('%Y%m%d'))
df['day_of_week'] = df.ActivityDate.apply(lambda x: x.strftime('%w'))
df.head()

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,ActivityDate,Time_str,day_of_week
0,1503960366,13162,25,13,328,728,1985,2016-04-12,2016-04-12,2
1,1503960366,10735,21,19,217,776,1797,2016-04-13,2016-04-13,3
2,1503960366,10460,30,11,181,1218,1776,2016-04-14,2016-04-14,4
3,1503960366,9762,29,34,209,726,1745,2016-04-15,2016-04-15,5
4,1503960366,12669,36,10,221,773,1863,2016-04-16,2016-04-16,6


In [90]:
sleep = read_file(data_csvs[-2])
sleep.SleepDay = pd.to_datetime(sleep['SleepDay'])
sleep.rename(columns={"SleepDay": "Day"}, inplace=True)
sleep.Day = sleep.Day.astype('str')
sleep.head()


../Data\sleepDay_merged.csv


,Id,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,1,327,346
1,1503960366,2016-04-13,2,384,407
2,1503960366,2016-04-15,1,412,442
3,1503960366,2016-04-16,2,340,367
4,1503960366,2016-04-17,1,700,712


In [91]:
sleep.dtypes

Id                     int64
Day                   object
TotalSleepRecords      int64
TotalMinutesAsleep     int64
TotalTimeInBed         int64
dtype: object

In [92]:
active =read_file(data_csvs[0])
active.ActivityDate = pd.to_datetime(active.ActivityDate)
active.ActivityDate = active.ActivityDate.astype('str')


active_df  =sliceDataBasedonDescribeval(active, 'std','ActivityDate')

Id_To_Idx = {id :num for num,id in enumerate(active.Id.unique())}
Idx_to_Id = {num :id for num,id in enumerate(active.Id.unique())}
active_df['PersonID'] = active_df.Id.map(Id_To_Idx)
active_df.PersonID =  active_df.PersonID.astype('str')
active_df



meged_active_sleep  = active_df.merge(sleep, left_on=["Id","ActivityDate" ], right_on=["Id", "Day"]).drop(columns = ['ActivityDate'])
meged_active_sleep

../Data\dailyActivity_merged.csv


<ipython-input-13-825045f40750>:166: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-92-572a506054e3>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

d:\Anaconda_2\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Time_str,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,13162,25,13,328,728,1985,2016-04-12,0,2016-04-12,1,327,346
1,1503960366,10735,21,19,217,776,1797,2016-04-13,0,2016-04-13,2,384,407
2,1503960366,9762,29,34,209,726,1745,2016-04-15,0,2016-04-15,1,412,442
3,1503960366,12669,36,10,221,773,1863,2016-04-16,0,2016-04-16,2,340,367
4,1503960366,9705,38,20,164,539,1728,2016-04-17,0,2016-04-17,1,700,712
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,8792009665,7174,10,20,301,749,2896,2016-04-30,31,2016-04-30,1,343,360
409,8792009665,1619,0,0,79,834,1962,2016-05-01,31,2016-05-01,1,503,527
410,8792009665,1831,0,0,101,916,2015,2016-05-02,31,2016-05-02,1,415,423
411,8792009665,2421,0,0,156,739,2297,2016-05-03,31,2016-05-03,1,516,545


In [93]:
meged_active_sleep

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Time_str,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,13162,25,13,328,728,1985,2016-04-12,0,2016-04-12,1,327,346
1,1503960366,10735,21,19,217,776,1797,2016-04-13,0,2016-04-13,2,384,407
2,1503960366,9762,29,34,209,726,1745,2016-04-15,0,2016-04-15,1,412,442
3,1503960366,12669,36,10,221,773,1863,2016-04-16,0,2016-04-16,2,340,367
4,1503960366,9705,38,20,164,539,1728,2016-04-17,0,2016-04-17,1,700,712
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,8792009665,7174,10,20,301,749,2896,2016-04-30,31,2016-04-30,1,343,360
409,8792009665,1619,0,0,79,834,1962,2016-05-01,31,2016-05-01,1,503,527
410,8792009665,1831,0,0,101,916,2015,2016-05-02,31,2016-05-02,1,415,423
411,8792009665,2421,0,0,156,739,2297,2016-05-03,31,2016-05-03,1,516,545


In [79]:
meged_active_sleep.head()

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Time_str,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,13162,25,13,328,728,1985,2016-04-12,0,2016-04-12,1,327,346
1,1503960366,10735,21,19,217,776,1797,2016-04-13,0,2016-04-13,2,384,407
2,1503960366,9762,29,34,209,726,1745,2016-04-15,0,2016-04-15,1,412,442
3,1503960366,12669,36,10,221,773,1863,2016-04-16,0,2016-04-16,2,340,367
4,1503960366,9705,38,20,164,539,1728,2016-04-17,0,2016-04-17,1,700,712


In [74]:
# df_ecdf = df[df["Calories"].isin(["DOG","CAT"])]
meged_active_sleep_gp = meged_active_sleep.groupby(['Id','Day'])[['TotalTimeInBed', 'Calories']].size()
fig_ecdf = px.line(meged_active_sleep, x="Calories", y="TotalTimeInBed", color="Id", markers=True)
fig_ecdf

In [85]:
df

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,ActivityDate,Time_str
0,1503960366,13162,25,13,328,728,1985,2016-04-12,2016-04-12
1,1503960366,10735,21,19,217,776,1797,2016-04-13,2016-04-13
2,1503960366,10460,30,11,181,1218,1776,2016-04-14,2016-04-14
3,1503960366,9762,29,34,209,726,1745,2016-04-15,2016-04-15
4,1503960366,12669,36,10,221,773,1863,2016-04-16,2016-04-16
...,...,...,...,...,...,...,...,...,...
935,8877689391,10686,17,4,245,1174,2847,2016-05-08,2016-05-08
936,8877689391,20226,73,19,217,1131,3710,2016-05-09,2016-05-09
937,8877689391,10733,18,11,224,1187,2832,2016-05-10,2016-05-10
938,8877689391,21420,88,12,213,1127,3832,2016-05-11,2016-05-11


In [95]:
           # pip install pandas

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1("What does the Body reveal", style={"textAlign":"center"}),
    html.Hr(),
    html.P("Choose Person of interest:"),
    html.Div(html.Div([
        dcc.Dropdown(id='Participant-type', clearable=False,
                     value="Std",
                     options=[{'label': x, 'value': x} for x in
                              df["Id"].unique()]),
    ],className="two columns"),className="row"),

    html.Div(id="output-div", children=[]),
])


@app.callback(Output(component_id="output-div", component_property="children"),
              Input(component_id="Participant-type", component_property="value"),
)
def make_graphs(Participant_chosen):
    # HISTOGRAM

    df_hist = df[df["Id"]==Participant_chosen].copy()
    # df_hist_group_day = df_hist.groupby(['Day', 'Id']).mean()
    fig_hist = px.histogram(df_hist, x="TotalSteps", color="day_of_week")
    fig_hist.update_xaxes(categoryorder="total descending")


    # # Scatter CHART 
    sleepActive = meged_active_sleep.copy()
    
    sleepActive_par = sleepActive[sleepActive.Id ==Participant_chosen]
    sleepActive_par['Id'] = sleepActive_par['Id'].astype('str')
    fig_strip= px.scatter(sleepActive_par, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')

    # Line per participant with calories
    fig_ecdf = px.line(sleepActive_par, x="TotalTimeInBed", y="Calories", color="Id", markers=True)


    # Scatter General All Participants
    sleepActive_general = meged_active_sleep.copy()
    sleepActive_general['Id'] = sleepActive_general['Id'].astype('str')

    scatterGeneral= px.scatter(sleepActive_general, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')

    # # SUNBURST
    # df_sburst = df.dropna(subset=['chip_status'])
    # df_sburst = df_sburst[df_sburst["intake_type"].isin(["STRAY", "FOSTER", "OWNER SURRENDER"])]
    # fig_sunburst = px.sunburst(df_sburst, path=["Participant_type", "intake_type", "chip_status"])

    # # Empirical Cumulative Distribution
    # df_ecdf = df[df["Participant_type"].isin(["DOG","CAT"])]
    # fig_ecdf = px.ecdf(df_ecdf, x="TotalTimeInBed", color="Std")

    # # LINE CHART
    # df_line = df.sort_values(by=["intake_time"], ascending=True)
    # df_line = df_line.groupby(
    #     ["intake_time", "Participant_type"]).size().reset_index(name="count")
    # print(df_line.head())
    # fig_line = px.line(df_line, x="intake_time", y="count",
    #                    color="Participant_type", markers=True)

    return [

        html.H2("Selected Participants", style={"textAlign":"center"}),
        html.Hr(),
            html.Div([
            html.Div([dcc.Graph(figure=fig_hist)], className="four columns"),
            html.Div([dcc.Graph(figure=fig_strip)], className="four columns"),
            html.Div([dcc.Graph(figure=fig_ecdf)], className="four columns"),
        ], className="row"),

        html.H2("All Participants", style={"textAlign":"center"}),
            html.Div([
            html.Div([dcc.Graph(figure=scatterGeneral)], className="six columns"),
            # html.Div([dcc.Graph(figure=fig_ecdf)], className="six columns"),
        ], className="row"),

    ]


if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2022 11:40:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2022 11:40:32] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 11:40:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2022 11:40:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
<ipython-input-95-ccaf709c57d1>:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [121]:
from jupyter_dash import JupyterDash     # pip install dash   
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud


# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions
url_coonections = "https://assets9.lottiefiles.com/private_files/lf30_5ttqPi.json"
url_companies = "https://assets9.lottiefiles.com/packages/lf20_EzPrWM.json"
url_msg_in = "https://assets9.lottiefiles.com/packages/lf20_8wREpI.json"
url_msg_out = "https://assets2.lottiefiles.com/packages/lf20_Cc8Bpg.json"
url_reactions = "https://assets2.lottiefiles.com/packages/lf20_nKwET0.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))


# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='D:/github_/DataAnalysisCourse/Idan/PcturesWeb/fitbit.jpg') # 150px by 45px
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Charming Data", target="_blank",
                                 href="https://youtube.com/charmingdata"
                    )
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        date=date(2018, 1, 1),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        date=date(2021, 4, 4),
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="67%", height="67%", url=url_coonections)),
                dbc.CardBody([
                    html.H6('Connections'),
                    html.H2(id='content-connections', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_companies)),
                dbc.CardBody([
                    html.H6('Companies'),
                    html.H2(id='content-companies', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_msg_in)),
                dbc.CardBody([
                    html.H6('Invites received'),
                    html.H2(id='content-msg-in', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                dbc.CardBody([
                    html.H6('Invites sent'),
                    html.H2(id='content-msg-out', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_reactions)),
                dbc.CardBody([
                    html.H6('Reactions'),
                    html.H2(id='content-reactions', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='bar-chart', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='TBD', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='pie-chart', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='wordcloud', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
], fluid=True)




datetime.date(2016, 4, 12)

In [159]:
from jupyter_dash import JupyterDash     # pip install dash   
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud


# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions
url_MeasureDays = "https://assets9.lottiefiles.com/private_files/lf30_5ttqPi.json"
url_caloriesBurned = "https://assets3.lottiefiles.com/packages/lf20_bdzc21c8.json"
url_sleep = "https://assets7.lottiefiles.com/private_files/lf30_em0dkba6.json"
url_PersonId = "https://assets9.lottiefiles.com/packages/lf20_dq0d7ah6.json"
url_Score = "https://assets4.lottiefiles.com/packages/lf20_rq3iw7zm.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))


# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='D:/github_/DataAnalysisCourse/Idan/PcturesWeb/Fbit.png', top=True) # 150px by 45px
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Data Set", target="_blank",
                                 href="https://github.com/idangrady/DataAnalysisCourse/tree/main/Data"
                    )
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        min_date_allowed= df.ActivityDate.min().date(),#date(2018, 1, 1),
                        max_date_allowed= df.ActivityDate.max().date(),
                        className='ml-5'
                    ),

                 dcc.Dropdown(id='Participant-type', 
                        clearable=False,
                        value="Id Participant",
                        options=[{'label': x, 'value': x} for x in
                        df["Id"].unique()],
                        className='mb-2 ml-2'
                    ),

                #     dcc.DatePickerSingle(
                #         id='my-date-picker-end',
                #         date=date(2021, 4, 4),
                #         className='mb-2 ml-2'
                #     ),

                # dcc.Dropdown(id='Participant-type', clearable=False,
                #      value="Std",
                #      options=[{'label': x, 'value': x} for x in
                #               df["Id"].unique()]),
                #     ],className="two columns"),className="row"),
                ])
            ], color="info"),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="60%", height="60%", url=url_MeasureDays)),
                dbc.CardBody([
                    html.H6('Measured Days'),
                    html.H2(id='Measured_Days', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="21%", height="21%", url=url_caloriesBurned)),
                dbc.CardBody([
                    html.H6('Total Calories Burned'),
                    html.H2(id='Calories_Burned', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="20%", height="20%", url=url_sleep)),
                dbc.CardBody([
                    html.H6('Average Sleep duration'),
                    html.H2(id='Ave_SleepDuration', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="35%", height="35%", url=url_PersonId)),
                dbc.CardBody([
                    html.H6('Perosn Id'),
                    html.H2(id='Perosn_Id', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_Score)),
                dbc.CardBody([
                    html.H6('Score'),
                    html.H2(id='Score_Prediction', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='bar-chart', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='TBD', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='pie-chart', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='wordcloud', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
], fluid=True)


if __name__=='__main__':
    app.run_server(debug=False, port=8000)

Dash app running on http://127.0.0.1:8000/
